In [11]:
import librosa
from librosa import display
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist
import numpy as np
import pickle
import os
import random
import python_speech_features
from speechCommon import *
from numba import jit

In [12]:
user = 'agoutam'
editTimeSec = 4
base_dir = "../ttemp/TamperingDetection"
hyp_dir = '../ttemp/TamperingDetection/hyp'
all_ids = '../ttemp/TamperingDetection/cfg_files/all.ids'
train_ids = '../ttemp/TamperingDetection/cfg_files/train.ids'
ref_dir = '../ttemp/TamperingDetection/speech/ref'
#validation_ids = '/home/{}/ttemp/TamperingDetection/data/cfg_files/validation.ids'.format(user)

### DTW

In [13]:
# inputs: 
# a query file with a specific edit type and piece number
# A set of DTW weights to use

# outputs:
# the cumulative cost matrix D, the best path through D, and the shape of D

def getPath(query_id, edit_type='n1', weightSet = 'D1'):
    mfcc_ref = readMFCC(base_dir, query_id, piece_type = 'reference')
    mfcc_query = readMFCC(base_dir, query_id, piece_type = 'queries', edit_type = edit_type, editTimeSec = editTimeSec)
    dist = euclidean_distances(mfcc_query, mfcc_ref)
    D, wp = librosa.sequence.dtw(C=dist, subseq=True, step_sizes_sigma=sigma, weights_mul=dtw_weights[weightSet])
    #return D, wp[::-1], dist.shape[1], dist.shape[0]
    return D, wp, dist.shape[1], dist.shape[0]

In [16]:
# inputs:
# query file with an edit type and a piece number
# set of DTW weights to use

# Outputs:
# Writes out a pickle file into the hypothesis directory 
# containing the best path, the cost matrix, and the shape of the cost matrix 
# for the alignment between a modified and unmodified query 

def alignAndSave(filename, query_id, editTimeSec, weightSet, edit_type, piece):
    if query_id == '00' or piece in [11, 12, 13]:
        return
    if not os.path.exists(filename):
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        mfcc_ref = readMFCC(base_dir, query_id, editTimeSec = editTimeSec)
        mfcc_query = readMFCC(base_dir, query_id, editTimeSec = editTimeSec, piece_type="queries", edit_type=edit_type+str(piece))
        dist = euclidean_distances(mfcc_query, mfcc_ref)
        D, wp = librosa.sequence.dtw(C=dist, subseq=True, step_sizes_sigma=sigma, weights_mul=dtw_weights[weightSet])
        costs = []
        for elem in wp:
            costs.append(dist[elem[0], elem[1]])
        res = {}
        res['wp'] = wp
        res['dist'] = np.asarray(costs)
        res['size'] = D.shape
        with open(filename, 'wb') as f:
            pickle.dump(res, f)

In [18]:
#Specify the benchmark and run on all data

benchmark = 'train'
with open ('/home/{}/ttemp/TamperingDetection/cfg_files/{}.ids'.format(user, benchmark), 'r') as f:
    for i, query_id in enumerate(f.read().split('\n')):
        if query_id == '':
            continue
        weightSet = 'D1'
        print(i, query_id, weightSet)
        for edit_type in ['i','r','d','n']:
            for piece in range(1, 11):
                filename = '{}/{}/{}sec/{}/{}_{}{}.pkl'.format(hyp_dir, benchmark, editTimeSec, weightSet, query_id, edit_type, piece)
                if not os.path.exists(filename):
                    print('aligning', query_id, edit_type, piece)
                    alignAndSave(filename, query_id, editTimeSec, weightSet, edit_type, piece)

0 10_6_17 D1
aligning 10_6_17 i 1
('../ttemp/TamperingDetection/hyp', 4, 'D1', '10_6_17', 'i', 1)
../ttemp/TamperingDetection/hyp/4sec/D1/10_6_17_i1.pkl
aligning 10_6_17 i 2
('../ttemp/TamperingDetection/hyp', 4, 'D1', '10_6_17', 'i', 2)
../ttemp/TamperingDetection/hyp/4sec/D1/10_6_17_i2.pkl
aligning 10_6_17 i 3
('../ttemp/TamperingDetection/hyp', 4, 'D1', '10_6_17', 'i', 3)
../ttemp/TamperingDetection/hyp/4sec/D1/10_6_17_i3.pkl
aligning 10_6_17 i 4
('../ttemp/TamperingDetection/hyp', 4, 'D1', '10_6_17', 'i', 4)
../ttemp/TamperingDetection/hyp/4sec/D1/10_6_17_i4.pkl
aligning 10_6_17 i 5
('../ttemp/TamperingDetection/hyp', 4, 'D1', '10_6_17', 'i', 5)
../ttemp/TamperingDetection/hyp/4sec/D1/10_6_17_i5.pkl
aligning 10_6_17 i 6
('../ttemp/TamperingDetection/hyp', 4, 'D1', '10_6_17', 'i', 6)
../ttemp/TamperingDetection/hyp/4sec/D1/10_6_17_i6.pkl
aligning 10_6_17 i 7
('../ttemp/TamperingDetection/hyp', 4, 'D1', '10_6_17', 'i', 7)
../ttemp/TamperingDetection/hyp/4sec/D1/10_6_17_i7.pkl
alignin

aligning 2_10_17 r 4
('../ttemp/TamperingDetection/hyp', 4, 'D1', '2_10_17', 'r', 4)
../ttemp/TamperingDetection/hyp/4sec/D1/2_10_17_r4.pkl
aligning 2_10_17 r 5
('../ttemp/TamperingDetection/hyp', 4, 'D1', '2_10_17', 'r', 5)
../ttemp/TamperingDetection/hyp/4sec/D1/2_10_17_r5.pkl
aligning 2_10_17 r 6
('../ttemp/TamperingDetection/hyp', 4, 'D1', '2_10_17', 'r', 6)
../ttemp/TamperingDetection/hyp/4sec/D1/2_10_17_r6.pkl
aligning 2_10_17 r 7
('../ttemp/TamperingDetection/hyp', 4, 'D1', '2_10_17', 'r', 7)
../ttemp/TamperingDetection/hyp/4sec/D1/2_10_17_r7.pkl
aligning 2_10_17 r 8
('../ttemp/TamperingDetection/hyp', 4, 'D1', '2_10_17', 'r', 8)
../ttemp/TamperingDetection/hyp/4sec/D1/2_10_17_r8.pkl
aligning 2_10_17 r 9
('../ttemp/TamperingDetection/hyp', 4, 'D1', '2_10_17', 'r', 9)
../ttemp/TamperingDetection/hyp/4sec/D1/2_10_17_r9.pkl
aligning 2_10_17 r 10
('../ttemp/TamperingDetection/hyp', 4, 'D1', '2_10_17', 'r', 10)
../ttemp/TamperingDetection/hyp/4sec/D1/2_10_17_r10.pkl
aligning 2_10_17 

aligning 3_31_18 r 2
('../ttemp/TamperingDetection/hyp', 4, 'D1', '3_31_18', 'r', 2)
../ttemp/TamperingDetection/hyp/4sec/D1/3_31_18_r2.pkl
aligning 3_31_18 r 3
('../ttemp/TamperingDetection/hyp', 4, 'D1', '3_31_18', 'r', 3)
../ttemp/TamperingDetection/hyp/4sec/D1/3_31_18_r3.pkl
aligning 3_31_18 r 4
('../ttemp/TamperingDetection/hyp', 4, 'D1', '3_31_18', 'r', 4)
../ttemp/TamperingDetection/hyp/4sec/D1/3_31_18_r4.pkl
aligning 3_31_18 r 5
('../ttemp/TamperingDetection/hyp', 4, 'D1', '3_31_18', 'r', 5)
../ttemp/TamperingDetection/hyp/4sec/D1/3_31_18_r5.pkl
aligning 3_31_18 r 6
('../ttemp/TamperingDetection/hyp', 4, 'D1', '3_31_18', 'r', 6)
../ttemp/TamperingDetection/hyp/4sec/D1/3_31_18_r6.pkl
aligning 3_31_18 r 7
('../ttemp/TamperingDetection/hyp', 4, 'D1', '3_31_18', 'r', 7)
../ttemp/TamperingDetection/hyp/4sec/D1/3_31_18_r7.pkl
aligning 3_31_18 r 8
('../ttemp/TamperingDetection/hyp', 4, 'D1', '3_31_18', 'r', 8)
../ttemp/TamperingDetection/hyp/4sec/D1/3_31_18_r8.pkl
aligning 3_31_18 r 9